<a href="https://colab.research.google.com/github/Prisha22/Airfoil-Optimization-Using-Gradient-Descent/blob/main/Gradient_descent_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Here the code is Using PARSEC parameterization and gets the optimized parameters to represent the airfoil (NACA 0018)
It then uses reults from XFOIL to update the parameters using weights and bias by numerical gradient descent method.

"""
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import os
from pyxfoil import Xfoil, set_workdir, set_xfoilexe
import pyxfoil
from scipy.optimize import least_squares
from pyxfoil import Xfoil, set_workdir, set_xfoilexe
import time
from scipy.linalg import solve
import math

xfoil_directory = 'path/to/xfoil/directory'
set_workdir(xfoil_directory)
set_xfoilexe(os.path.join(xfoil_directory, 'xfoil.exe'))
def fun_to_min(p):
        return  generate_airfoil_coordinates_parsec(xcoords, p) - ycoords
def objective(p):
    return fun_to_min(p)
def foil(p):
    if p is None:
        p=analysis()
    options = {
        'method': 'trf',
        'ftol': 1e-6,
        'max_nfev': 1200,
        'x_scale': [0.01, 0.01, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.01, 0.01, 2, 2],
        'diff_step': 0,
    }
    result = least_squares(fun_to_min, p, **options)
    para = result.x
    foil1 = generate_airfoil_coordinates_parsec(xcoords, para)
    return para

def analysis():

        ymax = max(ycoords)
        ymin = min(ycoords)
        xmax = max(xcoords)
        xmin = min(xcoords)

        ymax_index = np.argmax(ycoords)
        ymin_index = np.argmin(ycoords)
        x_maxy = xcoords[ymax_index]
        x_miny = xcoords[ymin_index]
        te_t = ycoords[0] - ycoords[-1]
        y_te = (ycoords[0] + ycoords[-1]) / 2


        x_up = xcoords[ymax_index - 1:ymax_index + 2]
        y_up = ycoords[ymax_index - 1:ymax_index + 2]
        coeffs_up = np.polyfit(x_up, y_up, 2)
        rUp = 1 / (2 * coeffs_up[0])


        x_low = xcoords[ymin_index - 1:ymin_index + 2]
        y_low = ycoords[ymin_index - 1:ymin_index + 2]
        coeffs_low = np.polyfit(x_low, y_low, 2)
        rlow = 1 / (2 * coeffs_low[0])


        slopeUp = 2 * coeffs_up[0]
        slopeLow = 2 * coeffs_low[0]


        det_value = abs((xcoords[1] - xcoords[0]) * (ycoords[-2] - ycoords[-1]) - (ycoords[1] - ycoords[0]) * (xcoords[-2] - xcoords[-1]))
        dot_product = (xcoords[1] - xcoords[0]) * (xcoords[-2] - xcoords[-1]) + (ycoords[1] - ycoords[0]) * (ycoords[-2] - ycoords[-1])
        beta = math.atan2(det_value, dot_product) * 180 / math.pi

        xx = np.zeros((5, 1))
        yy = np.zeros((5, 1))
        xx[1] = (xcoords[1] + xcoords[-2]) / 2
        xx[2] = (xcoords[0] + xcoords[-1]) / 2
        xx[3] = xx[1]
        yy[1] = (ycoords[1] + ycoords[-2]) / 2
        yy[2] = (ycoords[0] + ycoords[-1]) / 2
        yy[3] = xx[2]

        x1, y1 = xx[0], yy[0]
        x2, y2 = xx[1], yy[1]
        x3, y3 = xx[2], yy[2]

        det_value = abs((x1 - x2) * (y3 - y2) - (y1 - y2) * (x3 - x2))
        dot_product = (x1 - x2) * (x3 - x2) + (y1 - y2) * (y3 - y2)
        alpha = math.atan2(det_value, dot_product) * 180 / math.pi


        p = np.array([rUp, rlow, x_maxy, ymax, slopeUp, x_miny, ymin, slopeLow, te_t, y_te, alpha, beta])

        print("Parsec:",p)
        return p

def generate_airfoil_coordinates_parsec(x_coords, parsec_params):
    """
    Generates airfoil coordinates using the PARSEC parameterization.

    Args:
        x_coords (numpy.ndarray): Array of x-coordinates (0 to 1).
        parsec_params (numpy.ndarray): Array of 12 PARSEC parameters.

    Returns:
        numpy.ndarray: Array of (x, y) coordinates for the airfoil.
    """


    def parsec_for_fit_build(x,  p):


        locc = np.argmin(x)
        xUp = x[:locc]
        xLow = x[locc:]
        angle = -p[10] + p[11] / 2
        if not (-math.pi / 2 < angle < math.pi / 2):
            angle = np.mod(angle + np.pi / 2, np.pi) - np.pi / 2
        def Foil(x, aa):
            return aa[0] * x**(1/2) + aa[1] * x**(3/2) + aa[2] * x**(5/2) + aa[3] * x**(7/2) + aa[4] * x**(9/2) + aa[5] * x**(11/2)

        # Define matrices
        c1 = np.array([1, 1, 1, 1, 1, 1])
        c2 = np.array([p[2]**(1/2), p[2]**(3/2), p[2]**(5/2), p[2]**(7/2), p[2]**(9/2), p[2]**(11/2)])
        c3 = np.array([1/2, 3/2, 5/2, 7/2, 9/2, 11/2])
        c4 = np.array([(1/2) * p[2]**(-1/2), (3/2) * p[2]**(1/2), (5/2) * p[2]**(3/2),(7/2) * p[2]**(5/2), (9/2) * p[2]**(7/2), (11/2) * p[2]**(9/2)])
        c5 = np.array([(-1/4) * p[2]**(-3/2), (3/4) * p[2]**(-1/2), (15/4) * p[2]**(1/2), (35/4) * p[2]**(3/2), (63/4) * p[2]**(5/2), (99/4) * p[2]**(7/2)])
        c6 = np.array([1, 0, 0, 0, 0, 0])

        Cup = np.vstack((c1, c2, c3, c4, c5, c6))
        bup = np.array([p[9] +p[8]/2, p[3], np.tan(-p[10] - p[11] / 2), 0, p[4], math.sqrt(2 * abs(p[0]))])
        aup = solve(Cup, bup)
        foilUp = np.real(Foil(xUp, aup))

        c7 = np.array([1, 1, 1, 1, 1, 1])
        c8 = np.array([p[5]**(1/2), p[5]**(3/2), p[5]**(5/2), p[5]**(7/2), p[5]**(9/2), p[5]**(11/2)])
        c9 = np.array([1/2, 3/2, 5/2, 7/2, 9/2, 11/2])
        c10 = np.array([(1/2) * p[5]**(-1/2), (3/2) * p[5]**(1/2), (5/2) * p[5]**(3/2), (7/2) * p[5]**(5/2), (9/2) * p[5]**(7/2), (11/2) * p[5]**(9/2)])
        c11 = np.array([(-1/4) * p[5]**(-3/2), (3/4) * p[5]**(-1/2), (15/4) * p[5]**(1/2),(35/4) * p[5]**(3/2), (63/4) * p[5]**(5/2), (99/4) * p[5]**(7/2)])
        c12 = np.array([1, 0, 0, 0, 0, 0])

        Clo = np.vstack((c7, c8, c9, c10, c11, c12))
        angle = -p[10] + p[11] / 2
        blo = np.array([p[9] - p[8] / 2,p[6],np.tan(angle), 0, p[7],-math.sqrt(2 * abs(p[1]))])
        if not np.isfinite(Clo).all():
            raise ValueError("Clo contains inf or NaN values")
        if not np.isfinite(blo).all():
            raise ValueError("blo contains inf or NaN values")
        alower = solve(Clo, blo)
        foilLow = np.real(Foil(xLow,alower))
        foilUp[0]=foilLow[-1]
        foil = np.concatenate((foilUp, foilLow))
        return foil

    foil=parsec_for_fit_build(x_coords, parsec_params)
    return foil

def read_airfoil_coordinates(filename):
    """Reads airfoil coordinates from a file."""
    xcoords = []
    ycoords = []
    try:
        with open(filename, 'r') as f:
            for line in f:
                try:
                    x, y = map(float, line.strip().split())
                    xcoords.append(x)
                    ycoords.append(y)
                except ValueError:
                    pass
    except FileNotFoundError:
        print(f"Error: File not found at {filename}")
        return [], []
    return np.array(xcoords), np.array(ycoords)
def run_xfoil(airfoil_file, reynolds=100000):
    """
    Calculates the lift coefficient (CL) and drag coefficient (CD) using XFOIL through pyxfoil.

    Args:
        airfoil_coords (numpy.ndarray):  Numpy array of airfoil coordinates (x, y).
        Reynolds (float): Reynolds number.

    Returns:
        tuple: (CL, CD).  Returns (None, None) if XFOIL fails.
    """
    xfoil = Xfoil("NACA 0018")
    ali = alpha

    Re = reynolds
    xfoil.points_from_dat(airfoil_file)


    results = xfoil.run_polar(0, 4, 1, Re)


    #CLs,Cds,alphas=extract_values()
    return extract_values()
def extract_values():

    # Initialize lists for values
    alphas = []
    CLs = []
    Cds = []

    # Font configuration for plotting
    font = {'size': 6}
    plt.rc('font', **font)


    directory = "path\\to\\XFOIL\\Directory"


    polar_file = None
    for file in os.listdir(directory):
        if file.endswith('.pol'):
            polar_file = os.path.join(directory, file)
            print(f"Found polar file: {polar_file}")
            break


    if not polar_file:
        raise FileNotFoundError(f"No .pol file found in the directory '{directory}'.")

    with open(polar_file, 'r') as file:
        for line in file.readlines()[12:]:
            if line.startswith(' ') and not line.isspace():
                values = line.split()
                alphas.append(float(values[0]))
                CLs.append(float(values[1]))
                Cds.append(float(values[2]))

    plt.scatter(alphas, CLs, marker='o',label="XFoil")
    plt.xlabel('Alpha (degrees)')
    plt.ylabel('C L')


    # Placeholder lists for extracted data
    Cl_values = []
    Cd_values = []
    AoA_values = []
    return {
        'alpha':alphas,
        'cl':CLs,
        'cd':Cds,
        }

def cost_function_with_wb(omega, b,parsec_params, x_coords, desired_ld=20.0, alpha=4.0, reynolds=100000):
    """
    Cost function using omega and b to determine PARSEC parameters.
    """

    xcoords, ycoords = read_airfoil_coordinates("naca0018.dat")


    try:
        airfoil_file = "path/to/a/temp/storage/file/airfoil.dat"
        foil_coords = generate_airfoil_coordinates_parsec(x_coords, parsec_params)
        curve = np.column_stack((xcoords, foil_coords))
        np.savetxt(airfoil_file, curve, fmt='%.6f %.6f', header='n', comments='')
        results = run_xfoil(airfoil_file, reynolds)
        print(results)
        cl = results['cl'][-1]
        cd = results['cd'][-1]

        if cl is not None and cd is not None and cd > 0:
            ld_ratio = cl / cd
            cost = (desired_ld - ld_ratio)**2
            print(f'> omega={omega:.3f}, b={b:.3f}, L/D = {ld_ratio:.4f}, Cost = {cost:.4f}')
            return cost
        else:
            return 1e10

    except Exception as e:
        print(f"Error in cost_function_with_wb: {e}")
        return 1e10

def numerical_gradient_wb(func, omega, b, epsilon=1e-12, *args):
    grad_omega = (func(omega + epsilon, b,initial_parsec_params, *args) - func(omega - epsilon, initial_parsec_params,b, *args)) / (2*epsilon )
    grad_b = (func(omega, b + epsilon,initial_parsec_params, *args) - func(omega, b - epsilon,initial_parsec_params, *args)) / (2*epsilon )
    return grad_omega, grad_b

if __name__ == '__main__':

    xcoords, ycoords = read_airfoil_coordinates("naca0018.dat")


    omega = 0.0
    b = 0
    alpha = 1
    n_iterations = 100
    desired_ld_target = 20.0
    xcoords, ycoords = read_airfoil_coordinates("naca0018.dat")
    p=None

    initial_parsec_params = foil(p)
    print("iter\tomega\tb\tcost\t")
    for i in range(n_iterations):
        cost = cost_function_with_wb(omega, b, xcoords, desired_ld_target)
        grad_omega, grad_b = numerical_gradient_wb(cost_function_with_wb, omega, b, 1e-6, xcoords, desired_ld_target,initial_parsec_params)

        omega = omega - alpha * grad_omega
        b = b - alpha * grad_b
        gradient_weights = np.array([0.3, 0.1, 0.1, 0.1, 0.8, 0.2])

        print(f"{i}\t{omega:.3f}\t{b:.3f}\t{cost:.4f}\t")

        if i % 5 == 0:
          # HERE PARSEC PARAMETERS ARE UPDATED BASED ON THEIR SIGNIFICANCE TO THE OPTIMIZATION
          # THIS NEEDS TO BE CORRECTED HOWEVER I AM TOO TIRED CURRENTLY
            initial_parsec_params_base = initial_parsec_params
            optimized_parsec_params = np.copy(initial_parsec_params_base)
            optimized_parsec_params[0] = optimized_parsec_params[0] * 0.005 + 0.013
            #optimized_parsec_params[1] = optimized_parsec_params[1] * 0.05 + 0.27
            #optimized_parsec_params[2] = optimized_parsec_params[2] * 0.02 + 0.05
            #optimized_parsec_params[5] = optimized_parsec_params[3] * 0.02 - 0.05
            optimized_parsec_params[7] = optimized_parsec_params[4] * 0.01
            optimized_parsec_params[10] = optimized_parsec_params[5] * 2.0

            optimized_y_coords = generate_airfoil_coordinates_parsec(xcoords, optimized_parsec_params)
            plt.figure(figsize=(15, 6))
            plt.plot(xcoords, optimized_y_coords, 'b-', label=f'Iteration {i+1}')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.title('Optimized Airfoil Shape')
            plt.grid(True)
            plt.legend()

            plt.show()
        initial_parsec_params= optimized_parsec_params

